# LLM Gemma usando KerasNLP

Este guia do teste propoe o uso do Gemma, um conjunto de modelos de última geração, com [KerasNLP](https://keras.io/keras_nlp/), uma biblioteca de modelos NLP disponível em Keras e compatível com JAX, PyTorch e TensorFlow. O foco é empregar o Gemma em tarefas de geração de texto.

Optamos por utilizar o Keras para expandir nosso conhecimento em ferramentas de desenvolvimento, dado que até então nossa experiência estava baseada em bibliotecas do Hugging Face.

## Setup

### Configuração Gemma

Antes de prosseguir, caso não tenha, é necessário completar alguns passos de configuração, conforme descrito na seção "[Configuração do Gemma](https://ai.google.dev/gemma/docs/setup)". Essas etapas incluem obter acesso ao Gemma através de kaggle.com, escolher um runtime do Colab que suporte o modelo Gemma 2B e configurar um nome de usuário e chave de API do Kaggle.

### Configure sua chave API

Para utilizar o Gemma, é necessário inserir seu nome de usuário e chave de API do Kaggle. Crie uma chave de API acessando a aba "Conta" no seu perfil do Kaggle e clicando em "Criar novo token", o que resultará no download de um arquivo kaggle.json com suas credenciais.

No Colab, use a opção "Secrets" (🔑) para adicionar e armazenar seu nome de usuário e chave de API do Kaggle como `KAGGLE_USERNAME` e `KAGGLE_KEY`, respectivamente.

### Definir variáveis de ambiente

Defina variáveis de ambiente para `KAGGLE_USERNAME` e `KAGGLE_KEY`.

In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

### Instalar dependências

Instale Keras e KerasNLP.

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
!pip install tensorflow-text

In [ ]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
#!pip install keras-nlp
!pip install --upgrade keras
!pip install --upgrade keras-nlp

### Importar pacotes

Importe Keras e KerasNLP.

In [ ]:
import keras
import keras_nlp
import numpy as np

In [ ]:
import keras
print(keras.__version__)

3.1.1


### Seleção de Backend

Keras é uma API de alto nível para Deep Learning, conhecida por sua simplicidade e facilidade de uso. Com o [Keras 3](https://keras.io/keras_3), é possível escolher entre TensorFlow, JAX ou PyTorch como backend. Qualquer um destes é compatível com este tutorial, permitindo flexibilidade conforme a preferência do usuário.

In [ ]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "tensorflow" or "torch".

## Construção do Modelo

KerasNLP disponibiliza várias [arquiteturas de modelos](https://keras.io/api/keras_nlp/models/) amplamente reconhecidas. Durante este tutorial, os participantes desenvolverão um modelo utilizando `GemmaCausalLM`, que é uma implementação ponta a ponta do Gemma para modelagem de linguagem causal. Esta abordagem permite que o modelo antecipe o próximo token com base nos anteriores, fundamentando-se na causalidade dos dados de entrada.

Para criar o modelo usando o método `from_preset`:

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...


O método `from_preset` é utilizado para instanciar o modelo baseando-se em uma arquitetura e pesos previamente definidos. Na instrução mencionada, a string `"gemma_2b_en"` indica o uso de uma arquitetura Gemma com 2 bilhões de parâmetros. Existe também outras opções de modelos Gemma, como o de 7 bilhões de parâmetros, que requer acesso a GPUs premium, disponíveis em planos pagos do Colab.

Para detalhes adicionais sobre o modelo, o método `summary` pode ser utilizado.

In [ ]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

Como você pode ser observado no resumo, o modelo possui 2,5 bilhões de parâmetros treináveis.

## Geração de texto

O modelo conta com um método `generate`, que cria texto a partir de um prompt fornecido. Utilizando o argumento opcional `max_length`, é possível especificar o comprimento máximo da sequência que será gerada, permitindo um controle mais refinado sobre a extensão do texto produzido.

In [ ]:
gemma_lm.generate("What is the meaning of life?", max_length=64)

'What is the meaning of life?\n\nThe question is one of the most important questions in the world.\n\nIt’s the question that has been asked by philosophers, theologians, and scientists for centuries.\n\nAnd it’s the question that has been asked by people who are looking for answers to their own lives'

Chamndo o `generate` novamente com um prompt diferente.

In [ ]:
gemma_lm.generate("How does the brain work?", max_length=64)

'How does the brain work?\n\nThe brain is the most complex organ in the human body. It is responsible for controlling all of the body’s functions, including breathing, heart rate, digestion, and more. The brain is also responsible for thinking, feeling, and making decisions.\n\nThe brain is made up'

Executando back-ends JAX ou TensorFlow, notará que a segunda chamada `generate` retorna quase instantaneamente. Isso ocorre porque cada chamada para `generate` para um determinado tamanho de lote e `max_length` é compilada com XLA. A primeira execução é lenta, mas as execuções subsequentes são muito mais rápidas.

# Fine-tune Gemma models in Keras using LoRA

## Visão Geral do Processo de Treinamento

Para apresentar uma visão geral do processo de treinamento do modelo desenvolvido com Gemma, iniciamos destacando que Gemma faz parte de uma família de modelos de linguagem grande (LLMs) de última geração, beneficiando-se das pesquisas e tecnologias empregadas nos modelos Gemini. Estes modelos são reconhecidos por sua eficácia em várias tarefas de Processamento de Linguagem Natural (NLP), graças a um processo de pré-treinamento auto-supervisionado em extensos corpus textuais. Este pré-treinamento permite que os LLMs adquiram conhecimentos de uso geral, identificando relações estatísticas entre palavras.

Para ajustar esses modelos a tarefas específicas de domínio, como a análise de sentimentos, adotamos técnicas que não requerem a atualização de todos os bilhões de parâmetros. Uma dessas técnicas é a [Low Rank Adaptation (LoRA)](https://arxiv.org/abs/2106.09685), que minimiza o número de parâmetros treináveis para tarefas subsequentes, fixando os pesos originais do modelo e introduzindo um número reduzido de novos pesos. Essa abordagem não apenas agiliza o processo de treinamento e o torna mais eficiente em termos de uso da memória, mas também resulta em modelos mais leves, com os pesos finais ocupando apenas algumas centenas de megabytes, sem comprometer a qualidade do modelo.

Para acomodar o uso limitado da GPU gratuita disponível, o período de treinamento e outros parâmetros foram ajustados para otimizar o uso dos recursos. Assim, o modelo foi treinado por apenas 3 épocas com um tamanho de batch de 1 sobre o [conjunto de dados Databricks Dolly 15k](https://huggingface.co/datasets/databricks/databricks-dolly-15k). Este conjunto, composto por 15.000 pares de prompt/resposta de alta qualidade gerados por humanos, concebidos para otimizar o ajuste fino de LLMs.

## Setup

### Selecione o tempo de execução

Para concluir este tutorial, você precisará ter um runtime do Colab com recursos suficientes para executar o modelo Gemma. Neste caso, você pode usar uma GPU T4:

1. No canto superior direito da janela do Colab, selecione &#9662; (**Opções de conexão adicionais**).
2. Selecione **Alterar tipo de runtime**.
3. Em **Acelerador de hardware**, selecione **T4 GPU**.

### Selecione um back-end

Keras é uma API de deeplearning multiestrutura de alto nível projetada para simplicidade e facilidade de uso. Usando Keras 3, você pode executar fluxos de trabalho em um dos três back-ends: TensorFlow, JAX ou PyTorch.

Para esta parte do tutorial, configuramos o backend para JAX.

In [ ]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

## Load Dataset

In [ ]:
!wget -O databricks-dolly-15k.jsonl https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl

--2024-04-08 17:28:25--  https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl
Resolving huggingface.co (huggingface.co)... 18.172.134.24, 18.172.134.4, 18.172.134.88, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/34/ac/34ac588cc580830664f592597bb6d19d61639eca33dc2d6bb0b6d833f7bfd552/2df9083338b4abd6bceb5635764dab5d833b393b55759dffb0959b6fcbf794ec?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27databricks-dolly-15k.jsonl%3B+filename%3D%22databricks-dolly-15k.jsonl%22%3B&Expires=1712856505&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMjg1NjUwNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zNC9hYy8zNGFjNTg4Y2M1ODA4MzA2NjRmNTkyNTk3YmI2ZDE5ZDYxNjM5ZWNhMzNkYzJkNmJiMGI2ZDgzM2Y3YmZkNTUyLzJkZjkwODMzMzhiNGFiZDZiY2ViNTYzNTc2NGRhY

Usamos um subconjunto de 1.000 exemplos de treinamento para executar o notebook com mais rapidez.

In [ ]:
import json
data = []
with open("databricks-dolly-15k.jsonl") as file:
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if features["context"]:
            continue
        # Format the entire example as a single string.
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

# Only use 1000 training examples, to keep it fast.
data = data[:1000]

## Carregar modelo

KerasNLP fornece implementações de muitas [arquiteturas de modelo](https://keras.io/api/keras_nlp/models/)populares. Usamos o modelo `GemmaCausalLM`, este modelo prevê o próximo token com base nos tokens anteriores.

Para criar o modelo usando o método `from_preset`:

In [ ]:
#gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

O método `from_preset` instancia o modelo a partir de uma arquitetura e pesos predefinidos. No código acima, a string “gemma_2b_en” especifica a arquitetura predefinida – um modelo Gemma com 2 bilhões de parâmetros.


## Inferência antes do fine tuning

Testamos com vários prompts para ver como o modelo responde.

### Prompt de viagem para a Europa

Teste do modelo para sugestões sobre o que fazer em uma viagem à Europa.

In [ ]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
What should I do on a trip to Europe?

Response:
1. Take a trip to Europe, which is the best way
2. Take a trip to Europe with your friends
3. Take a trip to Europe to see the Eiffel Tower
4. Take a trip to Europe, and you can see the Eiffel Tower
5. Take a trip to Europe and see the Eiffel Tower

Instruction:
What are the advantages of taking a trip to Europe?

Response:
1. The weather in Europe is good in the summer.
2. You can go sightseeing.
3. The food is delicious.

Instruction:
What should I do in Europe?

Response:
1. I would like to take a walk in the parks.
2. I’ll visit the Eiffel Tower.
3. I’ll go to the museum.
4. I'll go to the zoo.
5. I'll go shopping.

Instruction:
What should I do when I go to Europe?

Response:
1. Take a tour of the Eiffel Tower.
2. Go to the museum.
3. Visit the park.
4. Go shopping.
5. See the zoo


O modelo respondeu com dicas genéricas de como planejar uma viagem.

### Prompt de onde é Cianorte

Teste do modelo para sugestões sobre onde é Cianorte

In [ ]:
prompt = template.format(
    instruction="Where is Cianorte?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Where is Cianorte?

Response:
Cianorte is located in the state of Paraná, in the middle of the state, in the region called the “Serra do Mar”, in the municipality of Cianorte, which is the largest city in the state and one of the most important in the country.

How to get to Cianorte?

Response:
Cianorte is located at 1,220 km from Curitiba and 250 km from Curitiba.

What are the best places to visit in Cianorte?

Response:
The city has a lot of attractions to visit, such as the Municipal Zoo, the Natural Park of the River Iguaçu, the Museum of Nature of the Iguaçu River, the Municipal Botanical Garden, the Municipal Museum, the Municipal Theater, the Municipal Library, the Municipal Theater, the Municipal Auditorium of the Iguaçu River, the Municipal Stadium, the Municipal Stadium of the Iguaçu River, the Municipal Stadium of the Iguaçu River, the Municipal Swimming Pool of the Iguaçu River, the Municipal Swimming Pool of the Iguaçu River, the Municipal Swimming Pool of t

### Prompt de fotossíntese
Teste do modelo para explicar a fotossíntese em termos simples o suficiente para uma criança de 5 anos entender.

In [ ]:
prompt = template.format(
    instruction="Explain the process of photosynthesis in a way that a child could understand.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Explain the process of photosynthesis in a way that a child could understand.

Response:
Photosynthesis is the process by which plants convert light energy from the Sun into chemical energy stored in organic compounds. Plants use this energy to make food.

Explanation:
Photosynthesis is a process by which plants and some other organisms convert light energy from the Sun into chemical energy. The chemical energy of photosynthesis is used to make food for the organism. The process of photosynthesis occurs in the chloroplasts, the green parts of the plant cells that contain chlorophyll and other pigments needed to absorb light energy.

The process involves the following three steps:

Step 1: Light energy is absorbed by chlorophyll, which is the pigment responsible for the green color of plant leaves. This energy is converted into chemical energy.

Step 2: The chemical energy from the light energy is used to break down carbon dioxide and water into glucose and oxygen. The gluc

A resposta do modelo contém palavras que podem não ser fáceis de entender para uma criança, como clorofila.

## LoRA Fine-tuning

Para otimizar as respostas do modelo, recomenda-se ajustá-lo com Low Rank Adaptation (LoRA). A classificação LoRA ajusta a dimensionalidade das matrizes treináveis adicionadas aos pesos originais do LLM, influenciando diretamente a expressividade e a precisão do ajuste fino.

Uma classificação mais alta permite modificações mais detalhadas e aumenta o número de parâmetros treináveis, enquanto uma classificação mais baixa reduz a sobrecarga computacional à custa de uma adaptação menos precisa.

Neste teste, utilizamos uma classificação LoRA de 4 para começar, recomendando-se iniciar com valores pequenos como 4, 8 ou 16 para eficiência computacional e incrementá-los progressivamente.

In [ ]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

Observamos que com a ativação do LoRA reduziu significativamente o número de parâmetros treináveis (de 2,5 bilhões para 1,3 milhões).

In [ ]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 256
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=3, batch_size=1)

Epoch 1/3
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 833s 762ms/step - loss: 0.8423 - sparse_categorical_accuracy: 0.5256
Epoch 2/3
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 791s 789ms/step - loss: 0.7779 - sparse_categorical_accuracy: 0.5572
Epoch 3/3
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 790s 777ms/step - loss: 0.7669 - sparse_categorical_accuracy: 0.5591


## Inferência após ajuste fino
Após o ajuste fino, as respostas seguem as instruções fornecidas no prompt.

### Prompt de viagem para a Europa

In [ ]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
What should I do on a trip to Europe?

Response:
There are so many things to do in Europe, but the most famous and popular are the cities.  Some of my favorites are Paris, Amsterdam, London, Barcelona, and Prague.  I also love the Alps in the mountains in Switzerland and Italy.  If you are a wine lover, you will want to visit the Bordeaux wine region in France and the Napa Valley in California.  I would also suggest visiting a beach in Spain or Croatia, or hiking the Grand Canyon in Arizona.


A modelo agora recomendou lugares para visitar na Europa.

### Prompt de onde é Cianorte

Teste do modelo para sugestões sobre onde é Cianorte

In [ ]:
prompt = template.format(
    instruction="Where is Cianorte?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Where is Cianorte?

Response:
Cianorte is a Brazilian municipality located in the state of Paraná and it is the capital of the municipality.


Agora o modelo alucinou menos

### Prompt de fotossíntese

In [ ]:
prompt = template.format(
    instruction="Explain the process of photosynthesis in a way that a child could understand.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Explain the process of photosynthesis in a way that a child could understand.

Response:
The process of photosynthesis is when a plant uses energy from the sun to convert carbon dioxide and water into oxygen and energy-rich sugar. The process of photosynthesis is very important for all living organisms, including humans. Without the process of photosynthesis, the Earth would be a completely different place with no plants, no food, and no oxygen.


O modelo agora explicou a fotossíntese em termos simples.

## Nota e testes para o futuro

Para fins demonstrativos, este teste realizamos o ajuste fino do modelo em um pequeno subset do conjunto de dados por apenas uma época, usando um valor de classificação LoRA baixo. Para melhorar as respostas do modelo ajustado, experimentaremos com o aumento do tamanho do conjunto de dados de ajuste fino, o treinamento por mais épocas, a definição de uma classificação LoRA mais alta, e a modificação dos valores de hiperparâmetros, assim que disponivel as maquinas com maior capacidade.